# Trying Different Methods of Reading PDF Files

In [118]:
file = 'Investasi/2-8-1995-UU-PASAR MODAL'

## Trying OCRMyPDF Library

In [1]:
import ocrmypdf
import pandas as pd

In [ ]:
if __name__ == '__main__':  # To ensure correct behavior on Windows and macOS
    ocrmypdf.ocr(file+'.pdf', file+'_OCR.pdf', output_type='pdf', deskew=True)

## Trying PDF Miner

In [ ]:
#!pip install pdfminer.six

In [102]:
from pdfminer.high_level import extract_text
text = extract_text(file+'.pdf')

## Trying PyPDF2 Library

In [50]:
#!pip install PyPDF2
from PyPDF2 import PdfReader
import re

In [120]:
pdf = PdfReader(open(file+'.pdf', "rb"))

In [ ]:
text = ""
for page_number in range(len(pdf.pages)): #len(pdf.pages)
    page = pdf.pages[page_number]
    text += page.extract_text()

### Conclusion
PyPDF2 is neater in comparison to PyMuPDF

## Trying to Clean Extracted Text

In [122]:
text = re.sub(r'\n', '', text)

'PRESIDENREPUBLIK INDONESIAUNDANG-UNDANG REPUBLIK INDONESIANOMOR 8 TAHUN 1995TENTANGPASAR MODALDENGAN RAHMAT TUHAN YANG MAHA ESAPRESIDEN REPUBLIK INDONESIA,Menimbang : a. bahwa tujuan pembangunan nasional adalah terciptanya suatumasyarakat adil dan makmur berdasarkan Pancasila danUndang-Undang Dasar 1945;b. bahwa Pasar Modal mempunyai peran yang strategis dalampembangunan nasional sebagai salah satu sumber pembiayaan bagidunia usaha dan wahana investasi bagi masyarakat;c. bahwa agar Pasar Modal dapat berkembang dibutuhkan adanyalandasan hukum yang kukuh untuk lebih menjamin kepastian hukumpihak-pihak yang melakukan kegiatan di Pasar Modal serta melindungikepentingan masyarakat pemodal dari praktik yang merugikan;d. bahwa sejalan dengan hasil-hasil yang dicapai pembangunan nasionalserta dalam rangka antisipasi atas globalisasi ekonomi,Undang-undang Nomor 15 Tahun 1952 tentang penetapanUndang-undang Darurat tentang Bursa (Lembaran Negara Tahun 1951Nomor 79) sebagai Undang-undang (Lembara

In [123]:
#!pip install nltk

In [ ]:
import nltk
#nltk.download('punkt')  # Download the required tokenizer data
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)

In [126]:
removal_index = []
for i, sentence in enumerate(sentences):
    if len(sentence)<40:
        removal_index.append(i)

In [ ]:
removal_index = sorted(removal_index, reverse=True)
for idx in removal_index:
   # checking whether the corresponding iterator index is less than the list length
   if idx < len(sentences):
      # removing element by index using pop() function
      sentences.pop(idx)

In [129]:
NO_PASAL = []
for i, sentence in enumerate(sentences):
    pasal = re.search(r'Pasal([ \d]*)', sentence)
    if pasal != None:
        NO_PASAL.append(pasal.group(0))
    elif i>0:
        NO_PASAL.append(NO_PASAL[i-1])
    else:
        NO_PASAL.append(pasal)

In [130]:
NO_PASAL_STRIPPED = []
for pasal in NO_PASAL:
    if pasal != None:
        pasal.strip()
        NO_PASAL_STRIPPED.append(pasal)

## Writing xlsx file with pasal/ayat level contents

In [131]:
def results_to_xlsx(NomorUrut, Isi, NamaFile):
    read_doc_results_df = pd.DataFrame(list(zip(NomorUrut, Isi)), columns = ['NomorUrut', 'Isi'])
    read_doc_results_df.to_excel(NamaFile+'.xlsx')

In [132]:
results_to_xlsx(NO_PASAL_STRIPPED, sentences, file)

## Looping PDF file Reading

In [141]:
documents_df = pd.read_excel('documents-info copy.xlsx')

In [136]:
documents_df.head()

,NamaFile,Hirarki,Nomor,Tahun,JenisPeraturan,Topik,NomorUrut,Isi
0,2-11-2016-UU-PENGAMPUNAN PAJAK.pdf,2,11,2016,UU,PENGAMPUNAN PAJAK,NaN,NaN
1,2-13-1962-UU-KETENTUAN POKOK BANK PEDA.pdf,2,13,1962,UU,KETENTUAN POKOK BANK PEDA,NaN,NaN
2,2-21-2008-UU-PERBANKAN SYARIAH.pdf,2,21,2008,UU,PERBANKAN SYARIAH,NaN,NaN
3,2-22-2006-UU-PERTANGGUNGJAWABAN ATAS PELAKSANA...,2,22,2006,UU,PERTANGGUNGJAWABAN ATAS PELAKSANAAN APBD,NaN,NaN
4,2-24-2004-UU-LEMBAGA PENJAMIN SIMPANAN.pdf,2,24,2004,UU,LEMBAGA PENJAMIN SIMPANAN,NaN,NaN


In [ ]:
for file in documents_df['NamaFile']:
    file = file.replace('.pdf', '')
    file = 'Investasi/'+file
    pdf = PdfReader(open(file+'.pdf', "rb"))
    text = ""
    for page_number in range(len(pdf.pages)): #len(pdf.pages)
        page = pdf.pages[page_number]
        text += page.extract_text()
    text = re.sub(r'\n', '', text)
    sentences = sent_tokenize(text)
    removal_index = []
    for i, sentence in enumerate(sentences):
        if len(sentence)<40:
            removal_index.append(i)
    removal_index = sorted(removal_index, reverse=True)
    for idx in removal_index:
        # checking whether the corresponding iterator index is less than the list length
        if idx < len(sentences):
        # removing element by index using pop() function
            sentences.pop(idx)
    NO_PASAL = []
    for i, sentence in enumerate(sentences):
        pasal = re.search(r'Pasal([ \d]*)', sentence)
        if pasal != None:
            NO_PASAL.append(pasal.group(0))
        elif i>0:
            NO_PASAL.append(NO_PASAL[i-1])
        else:
            NO_PASAL.append(pasal)
    NO_PASAL_STRIPPED = []
    for pasal in NO_PASAL:
        if pasal != None:
            pasal.strip()
            NO_PASAL_STRIPPED.append(pasal)
    results_to_xlsx(NO_PASAL_STRIPPED, sentences, file)
